In [1]:
import ee
import pandas as pd
from pathlib import Path
from datetime import datetime, timezone
import os
import re

from google.auth.transport.requests import AuthorizedSession

ee.Authenticate()  #  or !earthengine authenticate --auth_mode=gcloud
session = AuthorizedSession(ee.data.get_persistent_credentials())

session

## Create empty image collections as target
Create empty image collection with earthengine CLI
Purge contents if delete or clean repopulation needed
List COGs in GCS folder

```
cd /data/gis/raster_base/Alaska/AKVegMap/foliar_cover

conda activate ee

earthengine create folder projects/akveg-map/assets/foliar_cover
earthengine create collection projects/akveg-map/assets/foliar_cover/akveg_foliar_cover_v2p0

for i in `earthengine ls projects/akveg-map/assets/foliar_cover/akveg_foliar_cover_v2p0`; do earthengine rm $i; done

```

In [2]:
import json
# from urllib.parse import urlparse
import os
from pprint import pprint

In [3]:
# TODO Fix date extraction from filename to work for different filename formats
# Parts are zero-based
def load_gcs_cogs_to_collection(cogs, project_folder, collection, pft_part):
    # Request body as a dictionary.
    for cog in cogs['tif']:
      fileOnly = os.path.split(cog)[1]
      # print(fileOnly)

      cogName = fileOnly[:-4]
      print(cogName)
      
      # Split on both underscore and period
      parts = re.split(r'[_\.]', fileOnly)
      print(parts)
      
      pft = f"{parts[pft_part]}"
      print(pft)
        
      # start_dt = datetime(year, 1, 1)
      # start_dt = f"{start_dt:%Y-%m-%dT%H:%M:%SZ}"  # Format string for ISO 8601 + Z
      # end_dt = datetime(year + 1, 1, 1)
      # end_dt = f"{end_dt:%Y-%m-%dT%H:%M:%SZ}"  # Format string for ISO 8601 + Z
      # print(start_dt, end_dt)
        
      request = {
        'type': 'IMAGE',
        'gcs_location': {
          'uris': cog
        },
        'properties': {
            'pft': pft,
            'version': 2.0,
            #   'source': 'https://code.earthengine.google.com/067b10ee56537817756a3177a9138aee',
            # 'yyyymmdd_hhmmss': yyyymmdd_hhmmss
        },
        # 'startTime': start_dt,#'2023-01-01T00:00:00.000000000Z',
        # 'endTime': end_dt,#'2024-01-01T00:00:00.000000000Z',
      }

      # pprint(json.dumps(request))

      # A folder (or ImageCollection) name and the new asset name.
      asset_id = collection+'/'+cogName
      print(project_folder)
      print(asset_id)
        
      url = 'https://earthengine.googleapis.com/v1alpha/projects/{}/assets?assetId={}'
      print(url)

      response = session.post(
        url = url.format(project_folder, asset_id),
        data = json.dumps(request)
      )

      pprint(json.loads(response.content))
    print('done')


## Run it

In [4]:
# gsutil ls gs://akveg-data/rasters_cog/round_20241124_v20250502/*.tif > akveg_foliar_cover_v2p0_on_gcs.txt
project_folder = 'akveg-map'
collection = 'foliar_cover/akveg_foliar_cover_v2p0'
tif_txt = '/data/gis/raster_base/Alaska/AKVegMap/foliar_cover/akveg_foliar_cover_v2p0_on_gcs.txt'
cogs = pd.read_csv(tif_txt, header=None,names=['tif'])

load_gcs_cogs_to_collection(cogs, project_folder, collection, 0)


alnus_10m_3338
['alnus', '10m', '3338', 'tif']
alnus
akveg-map
foliar_cover/akveg_foliar_cover_v2p0/alnus_10m_3338
https://earthengine.googleapis.com/v1alpha/projects/{}/assets?assetId={}
{'bands': [{'dataType': {'precision': 'INT', 'range': {'max': 255}},
            'grid': {'affineTransform': {'scaleX': 10,
                                         'scaleY': -10,
                                         'translateX': -2180085,
                                         'translateY': 2388175},
                     'crsCode': 'EPSG:3338',
                     'dimensions': {'height': 198834, 'width': 307943}},
            'id': 'B0',
            'missingData': {'values': [255]},
            'pyramidingPolicy': 'MEAN'}],
 'cloudStorageLocation': {'uris': ['gs://akveg-data/rasters_cog/round_20241124_v20250502/alnus_10m_3338.tif']},
 'geometry': {'coordinates': [[172.20012497418128, 52.982186886504465],
                              [175.4747218234518, 48.999058895172126],
                 